In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12, extract_top_level_dict
from inner_loop_optimizers import LSLRGradientDescentLearningRule

In [89]:
#!pip install easydict
import easydict

In [90]:
args = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"alfa+maml",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.001,
  "meta_learning_rate":0.001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":48,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "attenuate": False,
  "alfa": True,
  "random_init": False,
  "backbone": "4-CONV"
}
)

device = torch.cuda.current_device()
im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104

In [91]:
# torch.ones(self.total_num_inner_loop_steps + 1) * self.init_learning_rate 확인
torch.ones(5 + 1) * 0.01

tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100])

### 1. meta_neural_network_architectures.py의 extract_top_level_dict 이 무엇일까?

In [92]:
current_dict = {"layer_dict.step3.linear1.weights'" : 1, 
                'layer_dict.step3.linear1.bias': 2}

extract_top_level_dict(current_dict)

{'step3': {"linear1.weights'": 1, 'linear1.bias': 2}}

### 2. ModuleDict으로 CNN 구현하기

In [95]:
class CNN_ModuleDict(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer_dict = nn.ModuleDict()
        
        self.build_network()
        
        for name, param in self.named_parameters():
            print(name, param.shape)
    
    def build_network(self):
        
        input_shape = (3,3,3)
        x = torch.zeros(3,3,3)

        out = x
        
        for i in range(5):
            self.layer_dict['conv{}'.format(i)] = nn.Conv2d(in_channels=out.shape[1],out_channels=6,kernel_size=5)
            
            # 이 코드가 왜 forward를 호출할까?
            out = self.layer_dict['conv{}'.format(i)](out)
            
    
    def forward(self, x):
        print("forward")
        

In [96]:
model = CNN_ModuleDict()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [6, 3, 5, 5], but got 3-dimensional input of size [3, 3, 3] instead